In [ ]:
!pip install playwright
!pip install nest_asyncio
!pip install openpyxl

In [0]:
!playwright install

In [0]:
dbutils.library.restartPython()

In [0]:
import asyncio
import csv
import os
import random

from playwright.async_api import async_playwright
import pandas as pd

NAME = '<NAME>'
WS = {'nitzan' : ['<web_socket_here>'],
        'shani' : ['<web_socket_here>'],
        'saar' : ['<web_socket_here>'],
        'shachaf': ['<web_socket_here']}



async def get_record(page):
    '''Extract job data from a single job page using Playwright'''

    # Initialize dictionary to store job data
    job_data = {}
    await page.mouse.move(random.randint(0, 500), random.randint(0, 500))
    await page.wait_for_timeout(random.uniform(1000, 3000))

    try:
        # Extract job title
        title_elem = await page.query_selector('h2[data-testid="simpler-jobTitle"]')
        job_data['title'] = await title_elem.inner_text()

    except Exception as e:
        return

    try:
        # Extract company name
        company_elem = await page.query_selector('a.jobsearch-JobInfoHeader-companyNameLink')
        job_data['company'] = await company_elem.inner_text()

    except Exception as e:
        return

    try:
        # Extract location
        location_elem = await page.query_selector('div[data-testid="jobsearch-JobInfoHeader-companyLocation"]')
        job_data['location'] = await location_elem.inner_text()

    except Exception as e:
        return

    # Extract pay, job type, work setting based on titles
    try:
        insights_elems = await page.query_selector_all('div.js-match-insights-provider-16m282m.e37uo190')
        for insight in insights_elems:
            title_elem = await insight.query_selector('h3.js-match-insights-provider-11n8e9a')
            if title_elem:
                title_text = await title_elem.inner_text()

                # Extract data based on the title
                if "Pay" in title_text:
                    salary_elems = await insight.query_selector_all('span.js-match-insights-provider-4pmm6z')
                    if salary_elems:
                        job_data['salary'] = await salary_elems[0].inner_text() if salary_elems else "N/A"
                elif "Job type" in title_text:
                    # Extract all job types if multiple
                    job_type_elems = await insight.query_selector_all(
                        'ul.js-match-insights-provider-h884c4 li span.js-match-insights-provider-4pmm6z')
                    if job_type_elems:
                        job_data['job_type'] = ', '.join(
                            [await job_type.inner_text() for job_type in job_type_elems]) if job_type_elems else "N/A"
                elif "Work setting" in title_text:
                    work_setting_elems = await insight.query_selector_all('span.js-match-insights-provider-4pmm6z')
                    if work_setting_elems:
                        job_data['work_setting'] = await work_setting_elems[
                            0].inner_text() if work_setting_elems else "N/A" #
                elif "Shift and schedule" in title_text:
                    shift_and_schedule_elems = await insight.query_selector_all('span.js-match-insights-provider-4pmm6z')
                    if shift_and_schedule_elems:
                        job_data['shift_and_schedule'] = await shift_and_schedule_elems[
                            0].inner_text() if shift_and_schedule_elems else "N/A" #
                elif "Medical specialty" in title_text:
                    medical_specialty_elems = await insight.query_selector_all('span.js-match-insights-provider-4pmm6z')
                    if medical_specialty_elems:
                        job_data['medical_specialty'] = await medical_specialty_elems[
                            0].inner_text() if medical_specialty_elems else "N/A"
                else:
                    # Extract any other information under different titles
                    other_info_elems = await insight.query_selector_all('span.js-match-insights-provider-4pmm6z')
                    if other_info_elems:
                        job_data['other_info'] = await other_info_elems[
                            0].inner_text() if other_info_elems else "N/A"

    except Exception as e:
        return


    try:
        # Extract job description
        job_description_elem = await page.query_selector('div.jobsearch-JobComponent-description')
        job_data['job_description'] = await job_description_elem.inner_text()

    except Exception as e:
        return

    return job_data

job_lock = asyncio.Lock()

# Assuming you have a list of job URLs
all_jobs = []

# Load existing results if the file exists
output_file = f'jobs_data_{NAME}.xlsx'
if os.path.exists(output_file):
    existing_data = pd.read_excel(output_file)
    all_jobs = existing_data.to_dict(orient='records')
else:
    existing_data = pd.DataFrame()

already_scraped = existing_data['job_url'].tolist() if not existing_data.empty else []
# get urls from file job_links.wlsx in the column Job URL
job_urls = pd.read_excel(f'job_links_{NAME}.xlsx')['Job URL'].tolist()
job_urls = list(set([link for link in job_urls if 'pagead' not in link]))
print("Total job URLs:", len(job_urls))
job_urls = [url for url in job_urls if url not in already_scraped]
print("Job URLs to scrape:", len(job_urls))


async def handle_job(p, idx, url):
    try:
        browser = await p.chromium.connect_over_cdp(WS[NAME][idx % len(WS[NAME])])
        # open browser without proxy
        #browser = await p.chromium.launch()
        context = await browser.new_context()
        page = await context.new_page()
        await page.goto(url)
        await page.mouse.move(random.randint(0, 500), random.randint(0, 500))
        await page.wait_for_timeout(random.uniform(5000, 20000))
        
        job_data = await get_record(page) # Scrape data from the job page
        if job_data:
            job_data['job_url'] = url  # Add the URL to the data dictionary
            async with job_lock:
                all_jobs.append(job_data)  # Store the job data
                new_data = pd.DataFrame(all_jobs)
                
                # Combine with existing data, ensuring no duplicates
                combined_data = pd.concat([existing_data, new_data]).drop_duplicates(subset=['job_url'], keep='last')

                # Save to Excel
                combined_data.to_excel(output_file, index=False)

            print(f"Scraped job {idx+1}/{len(job_urls)}")

        await browser.close()

    except Exception as e:
        print(f"Error scraping job {idx+1}: {e}")



async def main():
    ws_count = len(WS[NAME])
    print(f"Running {ws_count} workers")
    async with async_playwright() as p:
        for i in range(0, len(job_urls), ws_count):
            await asyncio.gather(*[handle_job(p, i + j, job_url) for j, job_url in enumerate(job_urls[i:i + ws_count])])

Total job URLs: 3806
Job URLs to scrape: 3492


In [ ]:
import nest_asyncio
nest_asyncio.apply()
asyncio.get_event_loop().run_until_complete(main())
